In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_train=pd.read_csv("E:\\jinnan\\train_after_clean.csv")
df_train=df_train[df_train['yield_rate']>0.87]
df_train.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
  
df_test=pd.read_csv("E:\\jinnan\\test_after_clean.csv")
df_test.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)

print(df_train.shape,df_test.shape)

(1381, 45) (150, 44)


In [3]:
target=df_train['yield_rate']
df_train.drop(columns=['yield_rate'],axis=1,inplace=True)
data=pd.concat([df_train,df_test],axis=0)
data["numerical_id"]=data['id'].apply(lambda x: int(x[7:]))
data.drop(columns=['id'],axis=1,inplace=True)
print(data.columns,data.shape)
raw_data_columns=data.columns

Index(['A1', 'A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A2',
       'A20', 'A20range', 'A21', 'A22', 'A24', 'A25', 'A26', 'A27', 'A28',
       'A28range', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B10',
       'B10range', 'B11', 'B11range', 'B12', 'B14', 'B2', 'B4', 'B4range',
       'B5', 'B6', 'B7', 'B8', 'B9', 'B9range', 'numerical_id'],
      dtype='object') (1531, 44)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
data['b14/a1_a3_a4_a19_b1_b12'] = data['B14']/(data['A1']+data['A3']+data['A4']+data['A19']+data['B1']+data['B12'])
data_collist=list(data.columns)
print(len(data_collist))
for col in raw_data_columns:
    data_collist.remove(col)
print(len(data_collist))

print(data.shape,df_train.shape,df_test.shape)
df_train=data[data_collist][:df_train.shape[0]]
df_test=data[data_collist][df_train.shape[0]:]
print(data.shape,df_train.shape,df_test.shape)


45
1
(1531, 45) (1381, 44) (150, 44)
(1531, 45) (1381, 1) (150, 1)


In [5]:
df_train.to_csv("E:\\jinnan\\unreason_feature_1_18_train.csv",index=False)
df_test.to_csv("E:\\jinnan\\unreason_feature_1_18_test.csv",index=False)

In [6]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [7]:
df_train=pd.read_csv("E:\\jinnan\\train_dispersion.csv")
df_train=df_train[df_train['yield_rate']>0.87]
df_train.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_train.columns:
    rate = df_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_train.drop(columns=[col], axis=1, inplace=True)
label_array=df_train["yield_rate"].values
df_train["id"]=df_train['id'].apply(lambda x: int(x[7:]))
df_train.drop(columns=['yield_rate'],axis=1,inplace=True)
print('train1:',df_train.shape)

df_train2=pd.read_csv("E:\\jinnan\\train_binning_mean.csv")
print('train2:',df_train2.shape)
df_train.index=range(df_train.shape[0])
df_train2.index=range(df_train2.shape[0])
df_train=pd.concat([df_train,df_train2],axis=1)

df_train3=pd.read_csv("E:\\jinnan\\multiply_feature_train.csv")
print('train3:',df_train3.shape)
df_train.index=range(df_train.shape[0])
df_train3.index=range(df_train3.shape[0])
df_train=pd.concat([df_train,df_train3],axis=1)

df_train4=pd.read_csv("E:\\jinnan\\add_minus_feature_train.csv")
print('train4:',df_train4.shape)
df_train.index=range(df_train.shape[0])
df_train4.index=range(df_train4.shape[0])
df_train=pd.concat([df_train,df_train4],axis=1)

df_train5=pd.read_csv("E:\\jinnan\\unreason_feature_1_18_train.csv")
print('train5:',df_train5.shape)
df_train.index=range(df_train.shape[0])
df_train5.index=range(df_train5.shape[0])
df_train=pd.concat([df_train,df_train5],axis=1)


# train_array=df_train[selected_col].values
train_array=df_train.values
print(df_train.columns,df_train.shape,train_array.shape)

df_test=pd.read_csv("E:\\jinnan\\test_dispersion.csv")
df_test.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_test.columns:
    rate = df_test[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_test.drop(columns=[col], axis=1, inplace=True)
columns_list=list(df_test.columns)
# columns_list.remove("id")
sample_id=df_test["id"]
df_test["id"]=df_test['id'].apply(lambda x: int(x[7:]))

df_test2=pd.read_csv("E:\\jinnan\\testA_binning_mean.csv")
df_test=pd.concat([df_test,df_test2],axis=1)

df_test3=pd.read_csv("E:\\jinnan\\multiply_feature_test.csv")
df_test=pd.concat([df_test,df_test3],axis=1)

df_test4=pd.read_csv("E:\\jinnan\\add_minus_feature_test.csv")
df_test=pd.concat([df_test,df_test4],axis=1)

df_test4=pd.read_csv("E:\\jinnan\\unreason_feature_1_18_test.csv")
df_test=pd.concat([df_test,df_test4],axis=1)
# test_array=df_test[selected_col].values
test_array=df_test.values
print(df_test.columns,df_test.shape,test_array.shape)

train1: (1381, 78)
train2: (1381, 105)
train3: (1381, 780)
train4: (1381, 272)
train5: (1381, 1)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B8minusB12', 'B8addB14', 'B8minusB14', 'B12addB12', 'B12minusB12',
       'B12addB14', 'B12minusB14', 'B14addB14', 'B14minusB14',
       'b14/a1_a3_a4_a19_b1_b12'],
      dtype='object', length=1236) (1381, 1236) (1381, 1236)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B8minusB12', 'B8addB14', 'B8minusB14', 'B12addB12', 'B12minusB12',
       'B12addB14', 'B12minusB14', 'B14addB14', 'B14minusB14',
       'b14/a1_a3_a4_a19_b1_b12'],
      dtype='object', length=1236) (150, 1236) (150, 1236)


In [8]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(train_array.shape[0])
predictions_lgb = np.zeros(test_array.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train_array[trn_idx], label_array[trn_idx])
    val_data = lgb.Dataset(train_array[val_idx], label_array[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train_array[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test_array, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, label_array)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000147975	valid_1's l2: 0.000175387
[400]	training's l2: 9.04199e-05	valid_1's l2: 0.000128338
[600]	training's l2: 7.0132e-05	valid_1's l2: 0.000117593
[800]	training's l2: 6.0563e-05	valid_1's l2: 0.000115029
[1000]	training's l2: 5.45281e-05	valid_1's l2: 0.000113646
[1200]	training's l2: 5.05137e-05	valid_1's l2: 0.000113071
[1400]	training's l2: 4.74975e-05	valid_1's l2: 0.000112657
Early stopping, best iteration is:
[1418]	training's l2: 4.72533e-05	valid_1's l2: 0.000112602
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.00014627	valid_1's l2: 0.00018462
[400]	training's l2: 8.69312e-05	valid_1's l2: 0.000157036
[600]	training's l2: 6.6854e-05	valid_1's l2: 0.000150826
[800]	training's l2: 5.72009e-05	valid_1's l2: 0.000149144
[1000]	training's l2: 5.12286e-05	valid_1's l2: 0.000148952
Early stopping, best iteration is:
[965]	training's l2

In [9]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_lgb})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_add_minus_unreason_lgb_1_18.csv",header=None,index=False)
print("mean_lgb:",np.mean(predictions_lgb))
print("var_lgb:",np.var(predictions_lgb))

mean_lgb: 0.9257243411575136
var_lgb: 0.0006523453109752762
